In [ ]:
# -*- coding: utf-8 -*-
#  Copyright 2023 United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Franck P. Vidal (UKRI-STFC)

In [ ]:
path = "/DATA/2023/DTHE/ZrO2-Cu-1mm-10umvx"

In [ ]:
import os


from DTHEDataReader import *
from cil.io import TIFFWriter
from cil.processors import TransmissionAbsorptionConverter, Slicer
from cil.recon import FDK
from cil.plugins.astra import FBP

from cil.utilities.display import show2D, show_geometry
from cil.utilities.jupyter import islicer, link_islicer

from cil.plugins.astra.operators import ProjectionOperator
from cil.optimisation.algorithms import CGLS, SIRT

In [ ]:
filename = os.path.join(path, "unireconstruction.xml")

reader = DTHEDataReader(file_name=filename)
data = reader.read()

In [ ]:
print(data)

In [ ]:
print(data.geometry)

CIL can illustrate the scan setup visually from the `AcquisitionData` geometry:

In [ ]:
show_geometry(data.geometry)

In [ ]:
islicer(data, direction='angle')

In [ ]:
data = TransmissionAbsorptionConverter()(data)

In [ ]:
islicer(data, direction='angle')

In [ ]:
data.reorder(order='tigre')

In [ ]:
ig = data.geometry.get_ImageGeometry()
fdk =  FDK(data, ig)
recons_FDK_tigre = fdk.run()

In [ ]:
visualisation_window = (0.0, 0.3)
islicer(recons_FDK_tigre, direction='vertical', minmax=visualisation_window)

In [ ]:
islicer(recons_FDK_tigre, direction='horizontal_x', minmax=visualisation_window)

We can save the reconstructed volume to disk for example as a stack of TIFFs:

In [ ]:
save_base_path = os.getcwd()
save_path = os.path.join(path, "recons_FDK_tigre")
print("Print the CT data will be saved in:", save_path)

if not os.path.isdir(save_path):
    os.makedirs(save_path)

TIFFWriter(data=recons_FDK_tigre, file_name=os.path.join(save_path, "out")).write()

In [ ]:
data.reorder('astra')
ig = data.geometry.get_ImageGeometry()
fbp =  FBP(ig, data.geometry)
recons_FDK_astra = fbp(data)


In [ ]:
save_base_path = os.getcwd()
save_path = os.path.join(path, "recons_FDK_astra")
print("Print the CT data will be saved in:", save_path)

if not os.path.isdir(save_path):
    os.makedirs(save_path)

TIFFWriter(data=recons_FDK_astra, file_name=os.path.join(save_path, "out")).write()

In [ ]:
sl1 = islicer(recons_FDK_tigre, minmax=visualisation_window)
sl2 = islicer(recons_FDK_astra, minmax=visualisation_window)
link_islicer(sl1, sl2)

In [ ]:
raise IOError("stop here")

In [ ]:
ag = data.geometry.copy()
A = ProjectionOperator(ig, ag, device="gpu")


In [ ]:
x0 = ig.allocate(0.0)


In [ ]:
mysirt_lower0 = SIRT(initial=x0, operator=A, data=data, max_iteration=1000, lower=0.0, update_objective_interval=50)

In [ ]:
mysirt_lower0.run(10, verbose=1)


In [ ]:
islicer(mysirt_lower0.solution, direction='vertical', minmax=visualisation_window)

In [ ]:
mysirt_lower0.run(100, verbose=1)
islicer(mysirt_lower0.solution, direction='vertical', minmax=visualisation_window)

In [ ]:
# setup CGLS
cgls = CGLS(initial=x0, 
            operator=A, 
            data=data,
            max_iteration = 10,
            update_objective_interval = 1 )

cgls.run(5, verbose=True)


In [ ]:
islicer(cgls.solution, direction='vertical', minmax=visualisation_window)

In [ ]:
cgls.run(100, verbose=1)
islicer(cgls.solution, direction='vertical', minmax=visualisation_window)